In [45]:
#include <iostream>
#include <ctime>
#include <cuda_runtime.h>
#include <curand_kernel.h>
#include "EasyBMP.h"

// Максимальная глубина рекурсии
#define RECURSION_DEPTH 5
// Проверка на ошибки возникающие в CUDA
#define checkCudaErrors(val) check_cuda( (val), #val, __FILE__, __LINE__ )

using namespace std;

// Функция вывода ошибок CUDA
void check_cuda(cudaError_t result, char const *const func, const char *const file, int const line) {
    if (result) {
        cerr << "CUDA error = " << static_cast<unsigned int>(result) << " at " <<
            file << ":" << line << " '" << func << "' \n";
        // Сброс устройства CUDA перед выходом
        cudaDeviceReset();
        exit(99);
    }
}

// Функция определяющая цвет полученной при столкновении луча и объекта
__device__ vec3 color(const ray& r, hitable **world, curandState *local_rand_state) {
    ray cur_ray = r;
    vec3 cur_attenuation = vec3(1, 1, 1);
    for(int i = 0; i < RECURSION_DEPTH; ++i) {
        hit_record rec;
        if ((*world)->hit(cur_ray, 0.001, FLT_MAX, rec)) {
            ray scattered;
            vec3 attenuation;
            if(rec.mat_ptr->scatter(cur_ray, rec, attenuation, scattered, local_rand_state)) {
                cur_attenuation *= attenuation;
                cur_ray = scattered;
            }
            else return vec3(0.0,0.0,0.0);
        }
        else {
            vec3 unit_direction = unit_vector(cur_ray.direction());
            float t = 0.5f*(unit_direction.y() + 1.0f);
            vec3 c = (1.0f-t)*vec3(1.0, 1.0, 1.0) + t*vec3(0.5, 0.7, 1.0);
            return cur_attenuation * c;
        }
    }
    return vec3(0.0,0.0,0.0); // Превышена глубина рекурсии
}

// Генерация случайных чисел на GPU
__global__ void rand_init(curandState *rand_state) {
    if (threadIdx.x == 0 && blockIdx.x == 0)
        curand_init(clock64(), 0, 0, rand_state); // Использование текущего времени как зерна
}

// Случайное заполнение пространства
__global__ void render_init(int max_x, int max_y, curandState *rand_state) {
    int i = threadIdx.x + blockIdx.x * blockDim.x;
    int j = threadIdx.y + blockIdx.y * blockDim.y;
    if((i >= max_x) || (j >= max_y)) return;
    int pixel_index = j*max_x + i;
    curand_init(1984 + pixel_index, 0, 0, &rand_state[pixel_index]);
}

// Рендеринг
global void render(vec3 *fb, int max_x, int max_y, int ns, camera **cam, hitable **world, curandState *rand_state) {
    // Определение индексов пикселей
    int i = threadIdx.x + blockIdx.x * blockDim.x;
    int j = threadIdx.y + blockIdx.y * blockDim.y;
    // Проверка выхода за пределы изображения
    if (i >= max_x || j >= max_y) return;
    // Индекс пикселя в буфере
    int pixel_index = j * max_x + i;
    // Инициализация локального состояния генератора случайных чисел
    curandState local_rand_state = rand_state[pixel_index];
    // Инициализация цвета пикселя
    vec3 col(0.0f, 0.0f, 0.0f);
    // Суммирование цветов для каждого подвыборки
    for (int s = 0; s < ns; ++s) {
        float u = (i + curand_uniform(&local_rand_state)) / static_cast<float>(max_x);
        float v = (j + curand_uniform(&local_rand_state)) / static_cast<float>(max_y);
        ray r = (*cam)->get_ray(u, v, &local_rand_state);
        col += color(r, world, &local_rand_state);
    }

    // Сохранение состояния генератора случайных чисел
    rand_state[pixel_index] = local_rand_state;
    // Нормализация цвета и применение гамма-коррекции
    col /= static_cast<float>(ns);
    col[0] = sqrt(col[0]);
    col[1] = sqrt(col[1]);
    col[2] = sqrt(col[2]);
    // Запись цвета в буфер изображения
    fb[pixel_index] = col;
}

// Создание сцены
global void create_world(hitable **d_list, hitable **d_world, camera **d_camera, int nx, int ny, curandState *rand_state) {
    if (threadIdx.x == 0 && blockIdx.x == 0) {
        curandState local_rand_state = *rand_state;
        int sphereCount = 0;
        // Создание большого фонового сферы
        d_list[sphereCount++] = new sphere(vec3(0, -1000, -1), 1000, new lambertian(vec3(0.5, 0.5, 0.5)));
        // Генерация случайных сфер
        for (int a = -1; a < 2; ++a) {
            for (int b = -1; b < 2; ++b) {
                float choose_mat = curand_uniform(&local_rand_state);
                float radius = curand_uniform(&local_rand_state);
                vec3 center(a + 5 * curand_uniform(&local_rand_state), radius, b + 3 * curand_uniform(&local_rand_state));
                if (choose_mat < 0.8) {
                    // Диффузный материал
                    d_list[sphereCount++] = createLambertianSphere(center, radius, local_rand_state);
                } else if (choose_mat < 0.95) {
                    // Металлический материал
                    d_list[sphereCount++] = createMetalSphere(center, radius, local_rand_state);
                } else {
                    // Стеклянный материал
                    d_list[sphereCount++] = new sphere(center, radius, new dielectric(1.5));
                }
            }
        }

        // Создание стеклянной сферы
        d_list[sphereCount++] = new sphere(vec3(0, 1, 0), 1, new dielectric(1.5));
        // Завершение создания списка объектов
        *d_world = new hitable_list(d_list, sphereCount);
        // Настройка камеры
        setupCamera(d_camera, nx, ny);
        *rand_state = local_rand_state; // Сохранение состояния генератора случайных чисел
    }
}

// Функция для создания диффузной сферы
hitable* createLambertianSphere(const vec3& center, float radius, curandState& rand_state) {
    vec3 albedo = vec3(curand_uniform(&rand_state), curand_uniform(&rand_state), curand_uniform(&rand_state));
    return new sphere(center, radius, new lambertian(albedo));
}

// Функция для создания металлической сферы
hitable* createMetalSphere(const vec3& center, float radius, curandState& rand_state) {
    vec3 albedo = vec3(0.5 * (1 + curand_uniform(&rand_state)),
                       0.5 * (1 + curand_uniform(&rand_state)),
                       0.5 * (1 + curand_uniform(&rand_state)));
    float fuzz = 0.5 * curand_uniform(&rand_state);
    return new sphere(center, radius, new metal(albedo, fuzz));
}

// Функция для настройки камеры
void setupCamera(camera **d_camera, int nx, int ny) {
    vec3 lookfrom(13, 2, 3);
    vec3 lookat(0, 0, 0);
    vec3 vup(0, 1, 0);
    float dist_to_focus = 10.0f;
    float aperture = 0.1f;
    *d_camera = new camera(lookfrom, lookat, vup, 30.0f, static_cast<float>(nx) / static_cast<float>(ny), aperture, dist_to_focus);
}


// Удаление всех элементов сцены с CPU
__global__ void free_world(hitable **d_list, hitable **d_world, camera **d_camera) {
    for(int i = 0; i < 22 * 22 + 1 + 3; ++i) {
        delete ((sphere *)d_list[i])->mat_ptr;
        delete d_list[i];
    }
    delete *d_world;
    delete *d_camera;
}


// Основная функция
int main() {
    // Параметры изображения
    const int nx = 1200;
    const int ny = 900;
    const int ns = 10; // Количество выборок на пиксель
    const int tx = 8;  // Ширина блока
    const int ty = 8;  // Высота блока
    cerr << "Rendering a " << nx << "x" << ny << " image with " << ns << " samples per pixel ";
    cerr << "in " << tx << "x" << ty << " blocks.\n";

    // Выделение памяти для буфера изображения
    size_t fb_size = nx * ny * sizeof(vec3);
    vec3 *fb;
    checkCudaErrors(cudaMallocManaged(&fb, fb_size));
    // Выделение памяти для состояния генератора случайных чисел
    curandState *d_rand_state;
    checkCudaErrors(cudaMalloc(&d_rand_state, nx * ny * sizeof(curandState)));
    curandState *d_rand_state2;
    checkCudaErrors(cudaMalloc(&d_rand_state2, sizeof(curandState)));
    // Инициализация генератора случайных чисел
    rand_init<<<1, 1>>>(d_rand_state2);
    checkCudaErrors(cudaGetLastError());
    checkCudaErrors(cudaDeviceSynchronize());
    // Выделение памяти для объектов сцены
    hitable **d_list;
    int num_hitables = 22 * 22 + 1 + 3;
    checkCudaErrors(cudaMalloc(&d_list, num_hitables * sizeof(hitable *)));
    hitable d_world;
    checkCudaErrors(cudaMalloc(&d_world, sizeof(hitable *)));
    camera d_camera;
    checkCudaErrors(cudaMalloc(&d_camera, sizeof(camera *)));
    // Создание сцены
    create_world<<<1, 1>>>(d_list, d_world, d_camera, nx, ny, d_rand_state2);
    checkCudaErrors(cudaGetLastError());
    checkCudaErrors(cudaDeviceSynchronize());
    // Начало отсчета времени рендеринга
    clock_t start = clock();
    // Определение размеров блоков и потоков для рендеринга
    dim3 blocks((nx + tx - 1) / tx, (ny + ty - 1) / ty);
    dim3 threads(tx, ty);
    // Инициализация рендеринга
    render_init<<<blocks, threads>>>(nx, ny, d_rand_state);
    checkCudaErrors(cudaGetLastError());
    checkCudaErrors(cudaDeviceSynchronize());
    // Запуск рендеринга
    render<<<blocks, threads>>>(fb, nx, ny, ns, d_camera, d_world, d_rand_state);
    checkCudaErrors(cudaGetLastError());
    checkCudaErrors(cudaDeviceSynchronize());
    // Остановка отсчета времени рендеринга
    clock_t stop = clock();
    double timer_seconds = static_cast<double>(stop - start) / CLOCKS_PER_SEC;
    cerr << "took " << timer_seconds << " seconds.\n";
    // Запись изображения в файл
    EasyBMP::RGBColor black(0, 0, 0);
    string file_path = "out.bmp";
    EasyBMP::Image img(nx, ny, file_path.c_str(), black);
    cerr << "P3\n" << nx << " " << ny << "\n255\n";
    // Заполнение изображения цветами из буфера
    for (int j = 0; j < ny; ++j) {
        for (int i = 0; i < nx; ++i) {
            size_t pixel_index = (ny - j - 1) * nx + i;
            int ir = static_cast<int>(255.99 * fb[pixel_index].r());
            int ig = static_cast<int>(255.99 * fb[pixel_index].g());
            int ib = static_cast<int>(255.99 * fb[pixel_index].b());
            img.SetPixel(i, j, EasyBMP::RGBColor(ir, ig, ib));
        }
    }
    img.Write();
    // Освобождение ресурсов
    free_world<<<1, 1>>>(d_list, d_world, d_camera);
    checkCudaErrors(cudaGetLastError());
    checkCudaErrors(cudaFree(d_camera));
    checkCudaErrors(cudaFree(d_world));
    checkCudaErrors(cudaFree(d_list));
    checkCudaErrors(cudaFree(d_rand_state));
    checkCudaErrors(cudaFree(d_rand_state2));
    checkCudaErrors(cudaFree(fb));
    cudaDeviceReset();
    return 0;
}

Rendering a 1200x900 image with 10 samples per pixel in 8x8 blocks.
took 0.106885 seconds.
P3
1200 900
255

